#### colab로 시작하기

<a href="https://colab.research.google.com/github/LDJWJ/00_TOTO_MLDL_CLASS/blob/master/DL04_02_GAN01_BASIC_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GAN(Generative Adversarial Networks)

### 딥러닝의 미래로 불리는 GAN은 대립(adversarial)하는 두 신경망을 경쟁시켜가며 결과물을 생성하는 방법이다.

### 실제이미지를 구분자(Discriminator)에게 이 이미지가 진짜임을 판단하게 하고, 생성자(Generator)를 통해 노이즈로부터 임의의 이미지를 만들고 이것을 다시 같은 구분자를 통해 진짜 이미지인지를 판단하게 한다.

### 응용 예 <br>

(가) 고흐 풍 그림으로 그려주기 <br>
(나) 선으로 그려진 만화를 자동으로 채색 <br>
(다) 모자이크를 없애주기 <br>
(라) GAN 기법을 이용한 자연어 문장 생성

### 이번시간의 학습 내용
* 손글씨 데이터 셋(MNIST)을 이용하여 손글씨를 무작위로 생성하는 예제 만들기

In [27]:
# https://arxiv.org/abs/1406.2661
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as ml

print("tf : {}, numpy : {}, matplotlib : {}".format(tf.__version__, 
                                                   ml.__version__, np.__version__))

tf : 1.5.0, numpy : 2.2.2, matplotlib : 1.18.0


In [28]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [39]:
print(mnist.test.images.shape)
print(mnist.test.labels.shape)
print(mnist.train.images.shape)
print(mnist.train.labels.shape)

(10000, 784)
(10000, 10)
(55000, 784)
(55000, 10)


In [57]:
print(type(mnist.train.images) )

<class 'numpy.ndarray'>


In [40]:
print(mnist.train.labels[0:10])

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


### 01. 기본 옵션 설정

In [42]:
total_epoch = 100   # epoch 수 설정
batch_size = 100    # 배치 사이즈
learning_rate = 0.0002  # 학습률

# 신경망 레이어 구성 옵션
n_hidden = 256          # 은닉층 노드
n_input = 28 * 28       # 입력 
n_noise = 128           # 생성기의 입력값으로 사용할 노이즈의 크기

### 02. 신경망 모델 구성
 * 노이즈를 이용하여 데이터 생성
 * 비지도학습이므로 Y가 없음

In [43]:
# GAN 도 Unsupervised 학습이므로 Autoencoder 처럼 Y 를 사용하지 않습니다.
X = tf.placeholder(tf.float32, [None, n_input])

# 노이즈 Z를 입력값으로 사용합니다.
Z = tf.placeholder(tf.float32, [None, n_noise])

### 생성자 신경망, 판별자 신경망 변수 선언

In [44]:
# 생성자 신경망에 사용하는 변수(Weight, bias)
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [45]:
# 판별기 신경망에 사용하는 변수들입니다.
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))

# 판별기의 최종 결과값은 얼마나 진짜와 가깝냐를 판단하는 한 개의 스칼라값입니다.
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

### 2-1 생성자(G) 신경망 구성
 * 무작위 생성한 노이즈를 받아, 가중치와 편향을 반영하여 은닉층 구성.
 * sigmoid 함수를 이용하여 최종 결과값 0~1 사이의 값 반환

In [46]:
def generator(noise_z):
    hidden = tf.nn.relu(
                    tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, G_W2) + G_b2)

    return output

### 2-2 구분자(D) 신경망 구성
 * 구분자 신경망 구성, 가중치와 편향을 반영한 데이터 출력
 * sigmoid 함수를 이용하여 최종 결과값 0~1 사이의 값 반환

In [47]:
def discriminator(inputs):
    hidden = tf.nn.relu(
                    tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, D_W2) + D_b2)

    return output

### 2-3 생성자 신경망의 노이즈 발생을 위한 노이즈 생성 함수

In [49]:
# 랜덤한 노이즈(Z)를 만듭니다.
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

### 2-4 신경망 구성

In [50]:
# 노이즈를 이용해 랜덤한 이미지를 생성합니다.
# Z에는 실행 시, noise가 입력됨.
G = generator(Z)
# 노이즈를 이용해 생성한 이미지가 진짜 이미지인지 판별한 값을 구합니다.
D_fake = discriminator(G)
# 진짜 이미지를 이용해 판별한 값을 구합니다.
D_real = discriminator(X)

 * GAN은 생성자(Generator) : 구분자가 1로 예측하도록 하는 것을 목표로 학습시킴. 
 * GAN은 구분자(Discriminator) : 진짜 데이터를 받으면 1로 가짜 데이터를 받으면 0으로 예측하도록 학습시킴.

### GAN의 모델의 최적화
* 각각의 신경망의 loss_G와 loss_D를 최대화 하는 것. 
   * 단, 서로의 손실이 연관되어 있어, 두 손실값이 같이 증가가 어려움.
* loss_D를 최대화하기 위해서는 D_gene값을 최소화시킴.
* 판별기에 진짜 이미지를 넣었을 때에도 최대값을 : tf.log(D_real)
* 가짜 이미지를 넣었을 때에도 최대값을 : tf.log(1 - D_gene)

In [51]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_fake))

* loss_G(생성자 손실)를 최대화하기 위해서는 D_gene값을 최대화 한다.
* 가짜 이미지를 넣었을 때, 판별기가 최대한 실제 이미지라고 판단하도록 생성기 신경망을 학습

In [52]:
# 결국 D_gene 값을 최대화하는 것이므로 다음과 같이 사용할 수 있습니다.
loss_G = tf.reduce_mean(tf.log(D_fake))

In [53]:
# loss_D 를 구할 때는 판별기 신경망에 사용되는 변수만 사용하고,
# loss_G 를 구할 때는 생성기 신경망에 사용되는 변수만 사용하여 최적화를 합니다.
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [54]:
# GAN 논문의 수식에 따르면 loss 를 극대화 해야하지만, minimize 하는 최적화 함수를 사용하기 때문에
# 최적화 하려는 loss_D 와 loss_G 에 음수 부호를 붙여줍니다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,
                                                         var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,
                                                         var_list=G_var_list)

### 03. 모델 학습

In [23]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

* 학습 후, 학습되는 이미지 저장을 위해 '/samples'라는 폴더 생성이 필요함.

In [26]:
%%time

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        # 판별기와 생성기 신경망을 각각 학습시킵니다.
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))
                                 
    #########
    # 학습이 되어가는 모습을 보기 위해 주기적으로 이미지를 생성하여 저장
    ######
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')

Epoch: 0000 D loss: -0.1653 G loss: -3.034
Epoch: 0001 D loss: -0.7106 G loss: -1.495
Epoch: 0002 D loss: -0.4771 G loss: -1.648
Epoch: 0003 D loss: -0.2223 G loss: -2.398
Epoch: 0004 D loss: -0.2874 G loss: -2.259
Epoch: 0005 D loss: -0.271 G loss: -2.542
Epoch: 0006 D loss: -0.1812 G loss: -2.933
Epoch: 0007 D loss: -0.2818 G loss: -2.744
Epoch: 0008 D loss: -0.3315 G loss: -2.738
Epoch: 0009 D loss: -0.4415 G loss: -2.354
Epoch: 0010 D loss: -0.4249 G loss: -2.392
Epoch: 0011 D loss: -0.5036 G loss: -2.21
Epoch: 0012 D loss: -0.3318 G loss: -2.419
Epoch: 0013 D loss: -0.3752 G loss: -2.35
Epoch: 0014 D loss: -0.3868 G loss: -2.449
Epoch: 0015 D loss: -0.2984 G loss: -2.649
Epoch: 0016 D loss: -0.3528 G loss: -2.587
Epoch: 0017 D loss: -0.4715 G loss: -2.755
Epoch: 0018 D loss: -0.4173 G loss: -2.33
Epoch: 0019 D loss: -0.4514 G loss: -2.615
Epoch: 0020 D loss: -0.3663 G loss: -2.427
Epoch: 0021 D loss: -0.4703 G loss: -2.258
Epoch: 0022 D loss: -0.5093 G loss: -2.449
Epoch: 0023 D l

### REF 
* GAN에 대한 논문 : https://arxiv.org/abs/1406.2661

Copyright 2020 LIM Co.(예영Edu Co.) all rights reserved. <br>
교육용으로 작성된 것으로 배포 및 복제시에 사전 허가가 필요합니다. <br>